# Scraper

In [ ]:
from extract.common import credentials
from extract.common import config

In [ ]:
import pymongo
from bson.json_util import dumps

host = config()['mongodb']['host']
user = credentials()['mongodb']['user']
password = credentials()['mongodb']['password']

class SaveProjects(object):
    def __init__(self):
        uri = "mongodb+srv://{}:{}@{}/test?retryWrites=true".format(user,password,host)
        client = pymongo.MongoClient(uri)
        self._db = client[config()['mongodb']['db']['name']]
    
    def save(self, projects):
        self._db.project.insert_many(projects)

In [ ]:
save_project = SaveProjects()

In [ ]:
save_project._db.project.delete_many({})

In [ ]:
save_project._db.project.estimated_document_count()
[i for i in save_project._db.project.find()]

In [ ]:
save_project.save([{"a":1},{"b":2}])

1825

In [21]:
from math import ceil
def year_to_periods(year, period_size):
    return ceil(365*year/period_size)
year_to_periods(4, 90)

17

In [2]:
from calculator.cumplo import predict_by_year
from calculator.cumplo import investment_return
from calculator.cumplo import fix_capital_no_extra
from calculator.cumplo import to_dataframe
from calculator.cumplo import predict_by_year_no_extra

In [26]:
vehicles = [{
    "time": 90,
    "yield": 0.18,
    "capital": 0
} for i in range(18)]
vehicles[0]['capital'] = 5000
prediction = predict_by_year_no_extra(vehicles)
prediction

{'time_serie': [{'time': 90,
   'extra_money': 0,
   'return': {'revenue': 225.0,
    'commission_percentage': 0.077328,
    'commission': 17.398799999999998,
    'return': 207.6012},
   'capital': 5207.6012},
  {'time': 180,
   'extra_money': 0,
   'return': {'revenue': 225.0,
    'commission_percentage': 0.077328,
    'commission': 17.398799999999998,
    'return': 207.6012},
   'capital': 5415.2024},
  {'time': 270,
   'extra_money': 0,
   'return': {'revenue': 225.0,
    'commission_percentage': 0.077328,
    'commission': 17.398799999999998,
    'return': 207.6012},
   'capital': 5622.8036},
  {'time': 360,
   'extra_money': 0,
   'return': {'revenue': 225.0,
    'commission_percentage': 0.077328,
    'commission': 17.398799999999998,
    'return': 207.6012},
   'capital': 5830.4048},
  {'time': 450,
   'extra_money': 0,
   'return': {'revenue': 225.0,
    'commission_percentage': 0.077328,
    'commission': 17.398799999999998,
    'return': 207.6012},
   'capital': 6038.006},
  {

In [27]:


import iplotter
from IPython.core.display import HTML
from visualize.chart import Chart

In [28]:
df = to_dataframe(prediction)
invested = (df['capital']-df['return']).tolist()
profits = df['return'].tolist()
x = df.index.strftime('%Y-%m-%d').tolist()

In [29]:
chart = Chart().stacked_area(x, profits, invested)
HTML(Chart.html)
c3_plotter = iplotter.C3Plotter()
c3_plotter.plot(chart)